In [17]:
from utils.data_io import read_json, export_json
from utils.query import *

In [18]:
sgpt_qald9_dbpedia_train = read_json("/upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9_dbpedia/train.json")

In [19]:
sgpt_qald9_dbpedia_test = read_json("/upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9_dbpedia/test.json")

In [20]:
sgpt_qald9_dbpedia_val = read_json("/upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9_dbpedia/val.json")

In [21]:
qald_9_wikidata_train = read_json("/upb/users/m/mengshim/profiles/unix/cs/MasterThesis_Mengshi/datasets/qald9plus/qald_9_plus_train_wikidata.json")["questions"]
qald_9_wikidata_test = read_json("/upb/users/m/mengshim/profiles/unix/cs/MasterThesis_Mengshi/datasets/qald9plus/qald_9_plus_test_wikidata.json")["questions"]

In [22]:
print(len(sgpt_qald9_dbpedia_train))

350


In [23]:
print(len(sgpt_qald9_dbpedia_val))

58


In [24]:
print(len(qald_9_wikidata_train))

371


In [25]:
print(sgpt_qald9_dbpedia_train[12])

{'id': '13', 'en_ques': 'give me the grandchildren of bruce lee.', 'en_key': 'bruce lee, grandchildren', 'de_ques': 'gib mir die enkel von bruce lee.', 'de_key': 'bruce lee, enkel', 'sparql': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?uri WHERE { res:Bruce_Lee dbo:child ?child . ?child <http://dbpedia.org/property/children> ?uri }', 'entities': ['Bruce_Lee'], 'relations': ['children', 'child'], 'fil_sparql': 'SELECT DISTINCT ?uri WHERE { res:Bruce_Lee dbo:child ?child . ?child dbp:children ?uri }', 'question': 'give me the grandchildren of bruce lee.', 'question_pos_tokens': ['VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PROPN', 'NOUN', 'X', 'PUNCT'], 'question_pos_ids': [7, 1, 1, 1, 1, 1, 2, 1, 5, 4], 'question_dep_ids': ['ROOT', 'dative', 'det', 'dobj', 'prep', 'compound', 'compound', 'pobj', 'pobj', 'punct'], 'question_dep_lvl': [1, 2, 3, 2, 3, 5, 5, 4, 4, 2], 'mapping': {'0': {'token': 'give', 'pos': 'VERB', 'dep-tok':

In [26]:
print([wikidata_entry for wikidata_entry in qald_9_wikidata_train if wikidata_entry["id"]==sgpt_qald9_dbpedia_train[0]["id"]])

[{'id': '1', 'question': [{'language': 'en', 'string': 'List all boardgames by GMT.'}, {'language': 'de', 'string': 'Liste die Brettspiele von GMT auf.'}, {'language': 'de', 'string': 'Zeige mir alle Brettspiele von GMT.'}, {'language': 'ru', 'string': 'Перечислите все игры GMT.'}, {'language': 'lt', 'string': 'Išvardinkite visus stalo žaidimus pagal GMT.'}, {'language': 'uk', 'string': 'Перерахуйте всі ігри GMT.'}, {'language': 'lt', 'string': 'Išvardykite visus GMT žaidimus.'}, {'language': 'fr', 'string': 'Listez tous les jeux de société de GMT.'}], 'query': {'sparql': 'PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P31 wd:Q131436 . }'}, 'answers': [{'head': {'vars': ['uri']}, 'results': {'bindings': [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q718'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1871'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/e

In [30]:
def create_id_dict(input_dict):
    return {entry['id']: entry for entry in input_dict}

def filter_sparql(qald_entry_wikidata):
    return postprocess_sparql(preprocess_sparql(qald_entry_wikidata["query"]["sparql"]))

def clean_question_string(qald_question_string):
    return re.sub(' +', ' ', qald_question_string.lower())

def is_question_match(clean_question_string, question_string, qald_question_string):
    return clean_question_string(qald_question_string)==clean_question_string(question_string)

def change_to_wikidata_sparql(sgpt_dataset, qald9_dataset):
    qald9_dict = create_id_dict(qald9_dataset)
    for sgpt_entry in sgpt_dataset:
        id: str = sgpt_entry['id']
        question_string: str = sgpt_entry["en_ques"]
        try: 
            qald_entry_wikidata: dict = qald9_dict[id]
            qald_question_string: str = qald_entry_wikidata["question"][0]["string"]
            if is_question_match(clean_question_string, question_string, qald_question_string):
                wikidata_sparql = filter_sparql(qald_entry_wikidata)
                sgpt_entry["fil_sparql"] = wikidata_sparql
            else:
                print(f"no qald entry for '{question_string}'")
        except Exception:
            sgpt_dataset.remove(sgpt_entry)
            print(f"no question {id}")
    return sgpt_dataset

sgpt_wikidata = change_to_wikidata_sparql(sgpt_qald9_dbpedia_train, qald_9_wikidata_train)
export_json("/upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9/train.json", sgpt_wikidata)

no question 6
no question 26
no question 30
no question 98
no question 146
no question 176
no question 208
no question 242
no question 246
no question 250
no question 276
no question 295
no qald entry for 'which holidays are celebrated around the world? test2'
no question 319
no question 323
no question 341
json file is exported to  /upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9/train.json


In [31]:
sgpt_wikidata = change_to_wikidata_sparql(sgpt_qald9_dbpedia_val, qald_9_wikidata_train)
export_json("/upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9/val.json", sgpt_wikidata)

no question 371
no question 387
no question 401
json file is exported to  /upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9/val.json


In [32]:
sgpt_wikidata = change_to_wikidata_sparql(sgpt_qald9_dbpedia_test, qald_9_wikidata_test)
export_json("/upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9/test.json", sgpt_wikidata)

no question 73
no question 155
no question 126
no question 83
no question 26
no question 14
json file is exported to  /upb/users/m/mengshim/profiles/unix/cs/SGPT-SPARQL-query-generation/data/qald9/test.json
